## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод
В исходных данных есть пропущенные значения в данных об общем трудовом стаже ('days_employed') и о ежемесячном доходе ('total_income') - 10% от датасета по этим параметрам отсутствуют. Это может быть связано с тем, что у некоторой категории клиентов нет подтверждения трудового стажа и официального дохода. Для данного проекта значения 'days_employed' не является ключевым. Поэтому будут заменеты медианным значением данного параметра по датасету. Параметр 'total_income' заявлен ключевым для данного иследования. Поэтому пропущенные значения будут заменены на медиану показателя для соответствующего типа занятости. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
def classified_dob_years(row):
    if (0 < row & row < 18):
        return 'несовершеннолетний!!!'
    if (18 <= row & row <= 25):
        return 'молодой (18 - 25 лет)'
    if (26 <= row & row <= 45):
        return 'взрослый (26 - 45 лет)'
    if (46 <= row & row <= 60):
        return 'старший (46 - 60 лет)'
    if row == 0:
        return 'возраст не указан'
    else:
        return 'старше 60'
    
def classified_days_employed(row):
    if row > 0:
        return 'на пенсии или безработный'
    if row < -15330:
        return 'работают на пенсии'
    
    return 'обычный стаж'


data['classified_days_employed'] = data['days_employed'].apply(classified_days_employed)
data['classified_dob_years'] = data['dob_years'].apply(classified_dob_years)

data_without_null = data.dropna()
data_null = data.loc[data['days_employed'].isnull()]
data_null['income_type'].value_counts()

income_employee = data_without_null.loc[data_without_null['income_type'] == 'сотрудник']['total_income'].median()
income_partner = data_without_null.loc[data_without_null['income_type'] == 'компаньон']['total_income'].median()
income_public_employee = data_without_null.loc[data_without_null['income_type'] == 'госслужащий']['total_income'].median()
income_pensionary = data_without_null.loc[data_without_null['income_type'] == 'пенсионер']['total_income'].median()
income_bissness_owner = data_without_null.loc[data_without_null['income_type'] == 'предприниматель']['total_income'].median()

data.loc[data['income_type'] == 'сотрудник','total_income'] = data.loc[data['income_type'] == 'сотрудник', 'total_income'].fillna(income_employee)
data.loc[data['income_type'] == 'компаньон', 'total_income'] = data.loc[data['income_type'] == 'компаньон', 'total_income'].fillna(income_partner)
data.loc[data['income_type'] == 'госслужащий', 'total_income'] = data.loc[data['income_type'] == 'госслужащий', 'total_income'].fillna(income_public_employee)
data.loc[data['income_type'] == 'пенсионер', 'total_income'] = data.loc[data['income_type'] == 'пенсионер', 'total_income'].fillna(income_pensionary)
data.loc[data['income_type'] == 'предприниматель','total_income'] = data.loc[data['income_type'] == 'предприниматель', 'total_income'].fillna(income_bissness_owner)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children                    21525 non-null int64
days_employed               19351 non-null float64
dob_years                   21525 non-null int64
education                   21525 non-null object
education_id                21525 non-null int64
family_status               21525 non-null object
family_status_id            21525 non-null int64
gender                      21525 non-null object
income_type                 21525 non-null object
debt                        21525 non-null int64
total_income                21525 non-null float64
purpose                     21525 non-null object
classified_days_employed    21525 non-null object
classified_dob_years        21525 non-null object
dtypes: float64(2), int64(5), object(7)
memory usage: 2.3+ MB


In [3]:
data_null = data.loc[data['days_employed'].isnull()]
days_employed_pensioner_type = data_without_null.loc[data_without_null['income_type'] == 'пенсионер']['days_employed'].median()
days_employed_26_45 = data_without_null.loc[data_without_null['classified_dob_years'] == 'взрослый (26 - 45 лет)']['days_employed'].median()
days_employed_46_60 = data_without_null.loc[data_without_null['classified_dob_years'] == 'старший (46 - 60 лет)']['days_employed'].median()
days_employed_18_25 = data_without_null.loc[data_without_null['classified_dob_years'] == 'молодой (18 - 25 лет)']['days_employed'].median()
days_employed_more_60 = data_without_null.loc[data_without_null['classified_dob_years'] == 'старше 60']['days_employed'].median()
days_employed_no_dob_years = data_without_null.loc[data_without_null['classified_dob_years'] == 'возраст не указан']['days_employed'].median()

data.loc[data['income_type'] == 'пенсионер','days_employed'] = data.loc[data['income_type'] == 'пенсионер', 'days_employed'].fillna(days_employed_pensioner_type)
data.loc[data['classified_dob_years'] == 'взрослый (26 - 45 лет)','days_employed'] = data.loc[data['classified_dob_years'] == 'взрослый (26 - 45 лет)', 'days_employed'].fillna(days_employed_26_45)
data.loc[data['classified_dob_years'] == 'старший (46 - 60 лет)','days_employed'] = data.loc[data['classified_dob_years'] == 'старший (46 - 60 лет)', 'days_employed'].fillna(days_employed_46_60)
data.loc[data['classified_dob_years'] == 'молодой (18 - 25 лет)','days_employed'] = data.loc[data['classified_dob_years'] == 'молодой (18 - 25 лет)', 'days_employed'].fillna(days_employed_18_25)
data.loc[data['classified_dob_years'] == 'старше 60','days_employed'] = data.loc[data['classified_dob_years'] == 'старше 60', 'days_employed'].fillna(days_employed_more_60)
data.loc[data['classified_dob_years'] == 'возраст не указан','days_employed'] = data.loc[data['classified_dob_years'] == 'возраст не указан', 'days_employed'].fillna(days_employed_no_dob_years)  
data['classified_days_employed'] = data['days_employed'].apply(classified_days_employed)
data['classified_dob_years'] = data['dob_years'].apply(classified_dob_years)
data_grouped = data.groupby(['income_type']).agg({'total_income' : 'median', 'days_employed' : 'median', 'debt' : ['count', 'sum']})

data.info()
print()
print(data_grouped)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children                    21525 non-null int64
days_employed               21525 non-null float64
dob_years                   21525 non-null int64
education                   21525 non-null object
education_id                21525 non-null int64
family_status               21525 non-null object
family_status_id            21525 non-null int64
gender                      21525 non-null object
income_type                 21525 non-null object
debt                        21525 non-null int64
total_income                21525 non-null float64
purpose                     21525 non-null object
classified_days_employed    21525 non-null object
classified_dob_years        21525 non-null object
dtypes: float64(2), int64(5), object(7)
memory usage: 2.3+ MB

                  total_income  days_employed   debt      
                        median         median  count   sum
income_type  

### Вывод
Больше половины значений (74%) в графе общего трудового стажа ('days_employed') содержат отрицательные значения. Следовательно, это не случайная ошибка в данных. Требуются пояснения о назначении знаков в этой графе для более корректного ввода пропущенных значений. 
Пропущенные значения в графе общего трудового стажа ('days_employed') заменены на медиану значений в данной графе в соответствии с возрастной категорией ('classified_dob_years'), поскольку трудовой стаж так или иначе связан с количеством прожитых лет. За исключением пенсионеров, поскольку их трудовой стаж видимо специально записывался большим положительным числом. Их трудовой стаж был посчитан как медиана трудового стажа пенсионеров ('income_type') в данной таблице.  
В графе возраста ('dob_years') есть значения 0. Либо данные клиента были введены не верно, были не введены вообще, возможно, были не подтверждены. 

### Замена типа данных

In [6]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children                    21525 non-null int64
days_employed               21525 non-null int64
dob_years                   21525 non-null int64
education                   21525 non-null object
education_id                21525 non-null int64
family_status               21525 non-null object
family_status_id            21525 non-null int64
gender                      21525 non-null object
income_type                 21525 non-null object
debt                        21525 non-null int64
total_income                21525 non-null int64
purpose                     21525 non-null object
classified_days_employed    21525 non-null object
classified_dob_years        21525 non-null object
dtypes: int64(7), object(7)
memory usage: 2.3+ MB


### Вывод
В датасете был изменен тип данных для колонок 'days_employed' и 'total income' методом astype() с типа float на тип int. 

### Обработка дубликатов

In [8]:
data.loc[data.duplicated(keep=False) == True].sort_values('dob_years')
data = data.drop_duplicates().reset_index(drop=True)
data.info()
data.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 14 columns):
children                    21471 non-null int64
days_employed               21471 non-null int64
dob_years                   21471 non-null int64
education                   21471 non-null object
education_id                21471 non-null int64
family_status               21471 non-null object
family_status_id            21471 non-null int64
gender                      21471 non-null object
income_type                 21471 non-null object
debt                        21471 non-null int64
total_income                21471 non-null int64
purpose                     21471 non-null object
classified_days_employed    21471 non-null object
classified_dob_years        21471 non-null object
dtypes: int64(7), object(7)
memory usage: 2.3+ MB


0

### Вывод
В исходных данных была найдена 71 полностью продублированная запись. Удалены методом drop_duplicates, чтобы избежать смещения статистики. Дублированные данные могут появляться при слиянии нескольких таблиц, или при случайном повторном внесении данных оператором. 

### Лемматизация

In [9]:
from pymystem3 import Mystem
m = Mystem()
     
data['lem_purpose'] = data['purpose'].apply(m.lemmatize)
data['lem_purpose'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             793
[на,  , проведение,  , свадьба, \n]                       773
[сыграть,  , свадьба, \n]                                 769
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           662
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 652
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            625
[покупка

### Вывод
Для лемматизации данных в графе 'Цель кредита' ('purpose') использована функция Mystem.lemmatize из библиотеки pymystem3. После лемматизации данных выделено несколько очевидных общих категорий целей: недвижимость и жилье, свадьба, автомобиль, образование. 

### Категоризация данных

In [15]:
def children_category(row):
    if row == 1:
        return '1 ребенок'
    if row == 2:
        return "2 ребенка"
    if row == 0:
        return '0 бездетные'
        
    return "многодетные"


def income_category(row):
    if row < 50000:
        return 'до 50000'
    if (row >= 50000) & (row < 100000):
        return '50000 - 100000'
    if (row >= 100000) & (row < 500000):
        return '100000 - 500000'
        
    return "свыше 500000"


def class_purpose(data):
    if 'недвижимость' in data:
        return 'недвижимость'
    if 'жилье' in data:
        return 'недвижимость'
    if 'свадьба' in data:
        return 'свадьба'
    if 'автомобиль' in data:
        return 'автомобиль'
    if 'oбразование,' or 'oбразование' in data:
        return 'образование'
    
    return 'другое'


data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['children_category'] = data['children'].apply(children_category)
children_category_vs_debt = data.pivot_table(index = 'children_category', values=['debt'], aggfunc=['count', 'sum'])
print(children_category_vs_debt)
print()

family_id_category = data.pivot_table(index=['family_status','family_status_id'], values=['debt'], aggfunc = ['count', 'sum'])
print(family_id_category.sort_values('family_status_id'))
print()

data['income_category'] = data['total_income'].apply(income_category)
total_income_vs_debt = data.pivot_table(index ='income_category', values=['debt'], aggfunc = ['count', 'sum'])
print(total_income_vs_debt)
print()

data['class_purpose'] = data['lem_purpose'].apply(class_purpose)
class_purpose_vs_debt = data.pivot_table(index ='class_purpose', values=['debt'], aggfunc = ['count', 'sum'])
print(class_purpose_vs_debt)

                   count   sum
                    debt  debt
children_category             
0 бездетные        14107  1063
1 ребенок           4856   445
2 ребенка           2128   202
многодетные          380    31

                                        count  sum
                                         debt debt
family_status         family_status_id            
женат / замужем       0                 12344  931
гражданский брак      1                  4163  388
вдовец / вдова        2                   959   63
в разводе             3                  1195   85
не женат / не замужем 4                  2810  274

                 count   sum
                  debt  debt
income_category             
100000 - 500000  16786  1373
50000 - 100000    4091   331
до 50000           372    23
свыше 500000       222    14

               count  sum
                debt debt
class_purpose            
автомобиль      4308  403
недвижимость   10814  782
образование     4014  370
свадьба      

### Вывод
В соответствии с целями данного исследования сформированы следующие критерии для категоризации данных: 
- количество детей ('children_category');
- семейное положение ('family_status_id'); 
- уровень дохода ('total_income_class); 
- основная цель кредита ('class_purpose'). 
По количеству детей клиенты с 3 и более детьми объединены в одну каnегорию - "многодетные", поскольку клиентов с 4 и 5 детьми значительно меньше, чем всех остальных. Отдельное их рассмотрение может привести к неправильнм выводам. 

В данных о детях содержались выбивающиеся значения: 76 человек с 20 детьми (вероятнее всего опечатка при внесении данных), 47 семей с '-1' ребенком. Произведена замена методом replace() - "-1" заменено на "1", "20" заменено на "2". Если данная замена не корректна, просьба внести пояснения для интерпретации таких данных. 
При заполнении графы об образовании не соблюдался единый формат записи, поэтому все записи об образовании приведены к единому формату - запись строчными буквами - методом str.lower().

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [17]:
children_category_vs_debt['debt_in_time_probability'] = 1 - (children_category_vs_debt['sum'] / children_category_vs_debt['count'])
print(children_category_vs_debt)

                   count   sum debt_in_time_probability
                    debt  debt                         
children_category                                      
0 бездетные        14107  1063                 0.924647
1 ребенок           4856   445                 0.908361
2 ребенка           2128   202                 0.905075
многодетные          380    31                 0.918421


### Вывод
После категоризации данных по количеству детей можно сделать следующие выводы:
1. Бездетные люди чаще берут кредит;
2. Бездетные люди чаще возвращают кредит (92,5% взятых кредитов вернули вовремя);
3. Возможно, что многодетные возвращают кредит вовремя чаще, чем клиенты, у которых 1 и 2 рабенка. Но скорее всего эта категория клиентов просто реже берет кредит.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [18]:
family_id_category['debt_in_time_probability'] = 1 - (family_id_category['sum'] / family_id_category['count'])
print(family_id_category.sort_values('debt_in_time_probability'))

                                        count  sum debt_in_time_probability
                                         debt debt                         
family_status         family_status_id                                     
не женат / не замужем 4                  2810  274                 0.902491
гражданский брак      1                  4163  388                 0.906798
женат / замужем       0                 12344  931                 0.924579
в разводе             3                  1195   85                 0.928870
вдовец / вдова        2                   959   63                 0.934307


### Вывод
После категоризации данных по семейному положению можно сделать следующие выводы:
1. Клиенты находящиеся или бывшие в официальном браке возвращают кредит вовремя чаще;
2. Не видно боьлшой разницы при возвращении кредита между неженатыми/незамужними и находящимися в гражданском браке;
3. Реже всего кредит берут вдовцы/вдовы и клиенты в разводе. С другой стороны они же чаще возвращают кредит вовремя. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [19]:
total_income_vs_debt['debt_in_time_probability'] = 1 - (total_income_vs_debt['sum'] / total_income_vs_debt['count'])
print(total_income_vs_debt)

                 count   sum debt_in_time_probability
                  debt  debt                         
income_category                                      
100000 - 500000  16786  1373                 0.918206
50000 - 100000    4091   331                 0.919091
до 50000           372    23                 0.938172
свыше 500000       222    14                 0.936937


### Вывод
При категоризации данных по ежемесячному уровню дохода можно сделать следующие выводы:
1. Больше всего кредитов берут клиенты с уровнем дохода от 100 до 500 т.р. в месяц;
2. Намного реже берут кредит клиенты с уровнем дохода до 50000 и свыше 500000;
3. Из сводной таблицы видно, что те, кто чаще берут кредит, реже его возвращают вовремя, но есть вероятность что это ошибка связанная с несбалансированными классами клиентов по ежемесячному доходу. 
4. Зависимость между уровнем охода и возвратом кредита в срок не явная, требуется больше данных для клиентов с низким уровнем дохода (менее 50000). (Клиенты со сверхвысоким уровнем дохода редко нуждаются в кредитах - могут позволить себе покупки за свой счет, не обременяя банк - сложнее найти выборку для исследования). 

- Как разные цели кредита влияют на его возврат в срок?

In [20]:
class_purpose_vs_debt['debt_in_time_probability'] = 1 - (class_purpose_vs_debt['sum'] / class_purpose_vs_debt['count'])
print(class_purpose_vs_debt.sort_values('debt_in_time_probability'))

               count  sum debt_in_time_probability
                debt debt                         
class_purpose                                     
автомобиль      4308  403                 0.906453
образование     4014  370                 0.907823
свадьба         2335  186                 0.920343
недвижимость   10814  782                 0.927686


### Вывод
При группировании данных по целям кредита можно сделать следующие выводы:
1. Чаще всего берут кредит на покупку недвижимости. Этот же тип кредита возвращают вовремя чаще всего;
2. Кредит на свадьбу в данной выборке брали реже всего, но возвращали его чаще, чем кредит на машину или образование;
3. Кредит на машину чаще всего возвращают с просрочками платежей. Возможно, это связано с тем, что это единственная категория движимого имущества (в данном датасете) - можно испортить, разбить, не получив ожидаемой пользы. 

### Шаг 4. Общий вывод
Исходя из полученных данных, можно сделать заключение:
1. Наиболее ответственными являются клиенты без детей, находящиеся или находившиеся в официальном браке, вне зависимости от уровня дохода. 
2. Самым возвращаемым кредитом является кредит на недвижимость (вероятность возврата кредита без задержек - 92,77%)
3. Клиенты без детей возвращают кредит вовремя чаще, чем клиенты с детьми (92,5%)
4. Клиенты находящиеся/находившиеся в официальном браке более дисциплинированы в том числе и в плане выплаты кредита (вероятность вернуть кредит вовремя выше 92,5%)
5. Из полученных данных не видно четкой зависимости между величиной ежемесячного дохода и возвратом кредита. Но в данных достаточно велика доля пропущенных значений для графы ежемесячного дохода, что может сильно влиять на выводы. 
6. При работе с данными нужны дополнительные комментарии к графе 'days_employed'. Трактовка этих значений неоднозначна, неясно, правильно ли заполнены пропущенные данные. 
7. В некоторых данных не соблюдена единая форма записи ('education', 'family_status') и допущены опечатки ('children'). Возможно, этого можно избежать введением единой формы заполнения данных, или использованием выпадающего списка.
8. В данных встречаются дубликаты (71 строка). Это может происходить при повторном введении данных оператором или при слиянии разных баз данных. 


